**Importa las bibliotecas necesarias.**

In [1]:
import psycopg2
from psycopg2 import Error # Importar la clase Error
from psycopg2 import sql
import pandas as pd

**Función para ejecutar una consulta SQL**

In [2]:
# Función para ejecutar una consulta SQL
def execute_query(conn, query, params=None):
    """
    Ejecuta una consulta SQL con manejo de errores y soporte para parámetros.
    """
    try:
        with conn.cursor() as cursor:
            cursor.execute(query, params)  # Ejecuta la consulta con o sin parámetros
            conn.commit()  # Confirma los cambios si no hay errores
            print("Query executed successfully")
            if cursor.description:
                # Si la consulta devuelve resultados, obtén los datos
                return cursor.fetchall()
    except psycopg2.Error as err:
        conn.rollback()  # Restaura la transacción si ocurre un error
        print(f"Error executing query: {err}")
        return None  # Devuelve None en caso de error


**Conexión a la base de datos de PostgreSQL data_powergen**

In [3]:
# Conexión a la base de datos
def create_database_connection():
    conn = None
    try:
        conn = psycopg2.connect(
            database="data_powergen", 
            user="datafundamental_owner", 
            password="uozci0wQGpA6", 
            host="ep-solitary-pond-a2p3evpo.eu-central-1.aws.neon.tech", 
            sslmode="require", 
            port=5432
        )
        print("Connected to 'data_powergen' successfully.")
    except psycopg2.Error as err:
        print(f"Error: {err}")
    return conn

**Crea la tabla resultados_join y la lista de consultas a ejecutar.**

In [4]:
sql_queries = {
    "create_resultados_join": """
        CREATE TABLE IF NOT EXISTS resultados_join AS
        SELECT 
            h.id_equipo,
            COUNT(h.id_orden) AS total_ordenes_mantenimiento,
            AVG(r.temperatura_c) AS promedio_temperatura,
            AVG(r.vibracion_mm_s) AS promedio_vibracion,
            AVG(r.presion_bar) AS promedio_presion
        FROM historicos_ordenes h
        LEFT JOIN registros_condiciones r
        ON h.id_equipo = r.id_equipo
        GROUP BY h.id_equipo;
    """,
    "total_ordenes_por_equipo": """
        SELECT id_equipo, COUNT(id_orden) AS total_ordenes
        FROM historicos_ordenes
        GROUP BY id_equipo;
    """,
    "duracion_promedio_vida_util": """
        SELECT AVG(vida_util_anios) AS duracion_promedio_vida_util
        FROM caracteristicas_equipos;
    """,
    "costo_promedio_correctivo": """
        SELECT AVG(costo_mantenimiento) AS costo_promedio_correctivo
        FROM historicos_ordenes
        WHERE tipo_mantenimiento = 'Correctivo';
    """,
    "equipo_mas_fallas": """
        SELECT id_equipo, COUNT(id_orden) AS total_correctivo
        FROM historicos_ordenes
        WHERE tipo_mantenimiento = 'Correctivo'
        GROUP BY id_equipo
        ORDER BY total_correctivo DESC
        LIMIT 1;
    """,
    "temperatura_resumen": """
        SELECT id_equipo, AVG(temperatura_c) AS promedio_temp, MAX(temperatura_c) AS max_temp
        FROM registros_condiciones
        GROUP BY id_equipo;
    """,
    "ordenes_correctivas_equipo": """
        SELECT id_orden, id_equipo, costo_mantenimiento, tiempo_reparacion_horas
        FROM historicos_ordenes
        WHERE tipo_mantenimiento = 'Correctivo' AND id_equipo = %s;
    """,
    "dias_vibracion_temperatura": """
        SELECT *
        FROM registros_condiciones
        WHERE vibracion_mm_s > 3.0 AND temperatura_c > 60;
    """,
    "tiempo_promedio_mantenimiento": """
        WITH diferencias AS (
            SELECT 
                id_equipo,
                EXTRACT(EPOCH FROM AGE(fecha, LAG(fecha) OVER (PARTITION BY id_equipo ORDER BY fecha))) / 3600 AS tiempo_entre_mantenimientos
            FROM historicos_ordenes
        )
        SELECT 
            id_equipo, 
            AVG(tiempo_entre_mantenimientos) AS tiempo_promedio_horas
        FROM diferencias
        GROUP BY id_equipo;
    """,
    "suma_horas_acumuladas": """
        SELECT id_equipo, SUM(horas_acumuladas) AS total_horas_acumuladas
        FROM registros_condiciones
        GROUP BY id_equipo;
    """
}



**Ejecuta el código (conectar y consultar)**

In [5]:
# Ejecución del script
if __name__ == "__main__":
    # Conectar a la base de datos
    db_conn = create_database_connection()

    # Ejecutar cada consulta
    for query_name, query in sql_queries.items():
        print(f"Executing query: {query_name}")
        result = execute_query(db_conn, query)

        equipo_id = 'E005'
        result = execute_query(db_conn, sql_queries["ordenes_correctivas_equipo"], (equipo_id,))
        print(result)



        if result:  # Si la consulta devuelve resultados, imprimirlos
            print(result)

    # Cerrar la conexión
    db_conn.close()


Connected to 'data_powergen' successfully.
Executing query: create_resultados_join
Query executed successfully
Query executed successfully
[('OT000013', 'E005', Decimal('639'), Decimal('18')), ('OT000119', 'E005', Decimal('3672'), Decimal('26')), ('OT000121', 'E005', Decimal('907'), Decimal('29')), ('OT000224', 'E005', Decimal('3001'), Decimal('43')), ('OT000225', 'E005', Decimal('1680'), Decimal('30')), ('OT000226', 'E005', Decimal('1126'), Decimal('36')), ('OT000260', 'E005', Decimal('2127'), Decimal('15')), ('OT000322', 'E005', Decimal('3998'), Decimal('15')), ('OT000411', 'E005', Decimal('3982'), Decimal('28')), ('OT000478', 'E005', Decimal('3399'), Decimal('22')), ('OT000479', 'E005', Decimal('4206'), Decimal('15')), ('OT000571', 'E005', Decimal('4235'), Decimal('28')), ('OT000593', 'E005', Decimal('967'), Decimal('13')), ('OT000598', 'E005', Decimal('2048'), Decimal('39')), ('OT000629', 'E005', Decimal('2294'), Decimal('24')), ('OT000639', 'E005', Decimal('2473'), Decimal('11')),